![MLU Logo](../../data/MLU_Logo.png)

# <a name="0">Machine Learning Accelerator - Computer Vision - Lecture 2</a>


## Customized Image Classification with AutoGluon

In this tutorial, we load images and the corresponding labels into [AutoGluon](https://autogluon.mxnet.io/index.html) and use this data to obtain a neural network that can classify new images. This is different from traditional machine learning where we need to manually define the neural network and then specify the hyperparameters in the training process. Instead, with just a single call to AutoGluon’s fit function, AutoGluon automatically trains many models with different hyperparameter configurations and returns the model that achieved the highest level of accuracy.

Note: Please use **GPU** for training. CPU training will lead to an unceasing running script. 

In [1]:
! pip install -q -r ../../requirements.txt

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p39/bin/python3.9 -m pip install --upgrade pip' command.


Let's import the ImagePredictor

In [2]:
from autogluon.vision import ImagePredictor, ImageDataset

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.1` and `torch==1.13.1+cu117` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '
INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmp3nqga8b_
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmp3nqga8b_/_remote_module_non_scriptable.py


To use AutoGluon for computer vision task training, we need to organize our data with the following structure:

    data/
    ├── train/
        ├── class1/
        ├── class2/
        ├── class3/
        ├── ...
    ├── test/
        ├── class1/
        ├── class2/
        ├── class3/
        ├── ...

Here each subfolder contains all images that belong to that category, e.g., `class1` contains all images belonging to the first class. We generally recommend at least 100 training images per class for reasonable classification performance, but this might depend on the type of images in your specific use-case.

## 1. <a name="1">Download the dataset</a>
(<a href="#0">Go to top</a>)

For demonstration purposes, we use a subset of the [Shopee-IET](https://www.kaggle.com/c/shopee-iet-machine-learning-competition/data) dataset from Kaggle. Each image in this data depicts a clothing item and the corresponding label specifies its clothing category. Our subset of the data contains the following possible labels: BabyPants, BabyShirt, womencasualshoes, womenchiffontop.

In [3]:
train_dataset, _, test_dataset = ImageDataset.from_folders('https://autogluon.s3.amazonaws.com/datasets/shopee-iet.zip')

data/
├── test/
└── train/


If you use this on your own dataset, just point it to your training or test folder. Example: `train_dataset = ImageDataset.from_folder('mydataset/train')`

Let's print the training dataset

In [4]:
print(train_dataset)

                                                 image  label
0    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
1    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
2    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
3    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
4    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
..                                                 ...    ...
795  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
796  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
797  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
798  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
799  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3

[800 rows x 2 columns]


## 2. <a name="2">Use AutoGluon to Fit Models</a>
(<a href="#0">Go to top</a>)

Now, let's fit a __classifier__ using AutoGluon [predictor.fit()](https://auto.gluon.ai/stable/tutorials/image_prediction/beginner.html). Within fit, the dataset is __automatically__ split into training and validation sets. The model with the best hyperparameter configuration is selected based on its performance on the __validation set__.

In [5]:
predictor = ImagePredictor()

time_limit = 10 * 60 # how long fit() should run (in seconds)
predictor.fit(train_dataset,
              time_limit=time_limit
             )

AutoGluon ImagePredictor will be deprecated in v0.7. Please use AutoGluon MultiModalPredictor instead for more functionalities and better support. Visit https://auto.gluon.ai/stable/tutorials/multimodal/index.html for more details! 
ImagePredictor sets accuracy as default eval_metric for classification problems.
Reset labels to [0, 1, 2, 3]
Randomly split train_data into train[720]/validation[80] splits.
The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1
Starting fit without HPO
INFO:TorchImageClassificationEstimator:modified configs(<old> != <new>): {
INFO:TorchImageClassificationEstimator:root.img_cls.model   resnet101 != resnet50
INFO:TorchImageClassificationEstimator:root.misc.seed       42 != 228
INFO:TorchImageClassificationEstimator:root.train.early_stop_patience -1 != 10
INFO:TorchImageClassificationEstimator:root.train.early_stop_max_value 1.0 != inf
INFO:TorchImageClassificationEstimator:root.train.early_stop_baseline 0.0 != -inf


## 3. <a name="3">Model Results</a>
(<a href="#0">Go to top</a>)

Use AutoGluon to Fit Models
Autogluon also provides the training results, which can be accessed by calling `predictor.fit_summary()`. 

In [6]:
fit_result = predictor.fit_summary()

In [7]:
fit_result

{'train_acc': 0.7972222222222223,
 'valid_acc': 0.875,
 'total_time': 116.24987030029297,
 'best_config': {'model': 'resnet50',
  'lr': 0.01,
  'epochs': 15,
  'batch_size': 16,
  'nthreads_per_trial': 128,
  'ngpus_per_trial': 8,
  'time_limits': 600,
  'dist_ip_addrs': None,
  'searcher': 'random',
  'early_stop_patience': 10,
  'early_stop_baseline': -inf,
  'early_stop_max_value': inf,
  'num_workers': 4,
  'gpus': [0],
  'seed': 228,
  'final_fit': False},
 'fit_history': {'train_acc': 0.7972222222222223,
  'valid_acc': 0.875,
  'total_time': 116.24987030029297,
  'best_config': {'model': 'resnet50',
   'lr': 0.01,
   'epochs': 15,
   'batch_size': 16,
   'nthreads_per_trial': 128,
   'ngpus_per_trial': 8,
   'time_limits': 600,
   'dist_ip_addrs': None,
   'searcher': 'random',
   'early_stop_patience': 10,
   'early_stop_baseline': -inf,
   'early_stop_max_value': inf,
   'num_workers': 4,
   'gpus': [0],
   'seed': 228,
   'final_fit': False}}}

We can access certain results from this summary. For example, training and validation accuracies below.

In [8]:
print('Train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))

Train acc: 0.797, val acc: 0.875


The best model and optimum hyperparameters: Learning rate, batch size, epochs can be printed with this:

In [9]:
fit_result['fit_history']['best_config']

{'model': 'resnet50',
 'lr': 0.01,
 'epochs': 15,
 'batch_size': 16,
 'nthreads_per_trial': 128,
 'ngpus_per_trial': 8,
 'time_limits': 600,
 'dist_ip_addrs': None,
 'searcher': 'random',
 'early_stop_patience': 10,
 'early_stop_baseline': -inf,
 'early_stop_max_value': inf,
 'num_workers': 4,
 'gpus': [0],
 'seed': 228,
 'final_fit': False}

 ## 4. <a name="4">Making Predictions</a>
(<a href="#0">Go to top</a>)

We can call the predict function to run on different images.

In [10]:
image_path = test_dataset.iloc[0]['image']
predictor.predict(image_path)

0    1
Name: label, dtype: int64

Let's get predictions on the test set.

In [11]:
pred = predictor.predict(test_dataset)
print(pred)

0     1
1     1
2     2
3     0
4     0
     ..
75    3
76    3
77    3
78    1
79    3
Name: label, Length: 80, dtype: int64
